# Scipt para buscar títulos da base de dados BB para a base de dados de CPB.


## 1) Carregamento e importação de bibliotecas e bases de dados 

In [ ]:
#Importa bibliotecas importantes
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import re
import os
import unidecode
import string
import nltk
import warnings
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
from pandas import ExcelWriter
from tqdm import tqdm


In [ ]:
#carrega Matchs de 2024
matchs_2024 = pd.read_excel('Bases/Matchs até 2024.xlsx', engine='openpyxl')

In [ ]:
# Carregando os datasets mais recentes da Ancine
Base_SAD = pd.read_excel('Bases/Obras Ancine.xlsx', engine='openpyxl')


In [ ]:
#Carregando base BB mais recente de 2025
Base_BB_import = pd.read_excel('Bases/BB Media - 2025-08-28 - Brazil - ALL PLATFORMS.xlsx', engine='openpyxl')


In [ ]:
#Carregando base BB já percorrida
Base_BB_2024 =  pd.read_excel('Bases/Base BB 2024 inicial.xlsx', engine='openpyxl')


In [ ]:
# ================================================================
# Resumo das bases carregadas
# ================================================================
bases = {
    'matchs_2024': matchs_2024,
    'Base_SAD': Base_SAD,
    'Base_BB': Base_BB_import,
    'Base_BB_2024': Base_BB_2024
}

print("Resumo das bases carregadas:\n")
for nome, df in bases.items():
    print(f"{nome:<15}: {len(df):>10,} linhas | {df.shape[1]:>3} colunas")


## 1.1) Limpeza de registros desnecessários na base da BB

In [ ]:
# Remover duplicatas dentro de Base_BB_1_filtrada, mantendo apenas o primeiro registro de cada UID
Base_BB_inicial = Base_BB_import.drop_duplicates(subset='BB UID')

# Contador de linhas final
print(f"Quantidade de linhas em Base_BB_inicial: {len(Base_BB_inicial)}")


In [ ]:
# Filtrar Base_BB removendo registros já existentes em Base_BB_2024
Base_BB_inicial = Base_BB_inicial[~Base_BB_inicial['BB UID'].isin(Base_BB_2024['UID'])].copy()

# Opcional: resetar o índice para limpeza visual
Base_BB_inicial.reset_index(drop=True, inplace=True)

print(f"Registros restantes em Base_BB_inicial: {len(Base_BB_inicial)}")


In [ ]:
# ================================================================
# Filtrando plataformas indesejadas da base Base_BB_inicial
# ================================================================

PLATAFORMAS_EXCLUIR = [
    'Archivio Luce', 'NBA League Pass', 'iQIYI', 'Zee5', 'FIFA+', 'Qello Concerts', 'ShemarooMe',
    'FlixOlé', 'Rakuten Viki', 'OnDemandKorea', 'Simply South', 'Digital Concert Hall', 'IWantTFC',
    'RT en Español', 'KOCOWA+', 'CINE.AR PLAY', 'IFI Archive Player', 'Hoichoi', 'Selecta TV',
    'Met Opera on Demand', 'Reel Short', 'American Indian Film Gallery', 'TVN Play', 'Teatrix',
    'Retina Latina', 'Demand Africa', 'DAZN', 'HispanTV', 'HENRI', 'Anime Onegai', 'ALTBalaji',
    'Digital Theatre', 'F1 TV'
]

# Contagem inicial
total_antes = len(Base_BB_inicial)

# Filtrando registros cujo campo 'Platform Name' está na lista
Base_BB_inicial = Base_BB_inicial[~Base_BB_inicial['Platform Name'].isin(PLATAFORMAS_EXCLUIR)].copy()

# Resetando índice
Base_BB_inicial.reset_index(drop=True, inplace=True)

# Contagem final e resumo
total_depois = len(Base_BB_inicial)
removidos = total_antes - total_depois

print(f"Total antes do filtro: {total_antes:,}")
print(f"Total após o filtro : {total_depois:,}")
print(f"Registros removidos : {removidos:,}")


## 1.2) Resumo Estatístico dos dados

In [ ]:
# ================================================================
# Resumo estatístico e estrutural das bases Base_SAD e Base_BB_inicial
# ================================================================

def resumo_dataframe(df, nome_df):
    """
    Exibe resumo de atributos e estatísticas descritivas de um DataFrame
    em formato amigável e legível no JupyterLab.
    """
    display(HTML(f"<h3 style='color:#2a5599;'>📊 Resumo da base <code>{nome_df}</code></h3>"))

    # === 1️⃣ Estrutura e tipos ===
    info_df = pd.DataFrame({
        "Coluna": df.columns,
        "Tipo de Dado": [str(df[col].dtype) for col in df.columns],
        "Valores Não Nulos": [df[col].notna().sum() for col in df.columns],
        "Valores Nulos": [df[col].isna().sum() for col in df.columns],
        "% Nulos": [df[col].isna().mean() * 100 for col in df.columns]
    })

    styled_info = (
        info_df.style
        .format({'% Nulos': '{:.2f}%'})
        .set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
            {'selector': 'td', 'props': [('padding', '6px 12px')]},
        ])
        .set_properties(subset=['Coluna'], **{'text-align': 'left'})
        .set_properties(subset=['Tipo de Dado'], **{'text-align': 'center'})
        .set_properties(subset=['Valores Não Nulos', 'Valores Nulos', '% Nulos'], **{'text-align': 'right'})
        .hide(axis="index")
    )
    display(HTML("<h4>📋 Estrutura e Qualidade dos Dados</h4>"))
    display(styled_info)

    # === 2️⃣ Estatísticas descritivas ===
    desc = df.describe(include='all').transpose().fillna("")
    styled_desc = (
        desc.style
        .set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
            {'selector': 'td', 'props': [('padding', '6px 12px')]},
        ])
        .set_properties(**{'text-align': 'right'})
    )
    display(HTML("<h4>📈 Resumo Estatístico</h4>"))
    display(styled_desc)
    display(HTML("<hr style='margin:25px 0;'>"))

# ================================================================
# Aplicação da função às suas bases
# ================================================================
resumo_dataframe(Base_SAD, "Base_SAD")
resumo_dataframe(Base_BB_inicial, "Base_BB_inicial")


In [ ]:
# ================================================================
# Amostra das primeiras linhas (head) das bases Base_SAD e Base_BB_inicial
# ================================================================

def exibir_head(df, nome_df, linhas=10):
    """
    Exibe as primeiras linhas de um DataFrame (head) com formatação
    legível no JupyterLab, no estilo de tabela do Colab.
    """
    display(HTML(f"<h3 style='color:#2a5599;'>🔎 Amostra das {linhas} primeiras linhas da base <code>{nome_df}</code></h3>"))

    styled_head = (
        df.head(linhas)
        .style
        .set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
            {'selector': 'td', 'props': [('padding', '6px 12px')]},
        ])
    )

    display(styled_head)
    display(HTML("<hr style='margin:25px 0;'>"))

# ================================================================
# Aplicação da função às duas bases
# ================================================================
exibir_head(Base_SAD, "Base_SAD", linhas=10)
exibir_head(Base_BB_inicial, "Base_BB_inicial", linhas=10)


## 2) Pré-processamento e limpeza inicial de dados e criação de dataframes separados

## 2.1) Trimming do dataframe da BB

In [ ]:
# ================================================================
# Selecionar apenas as colunas de interesse da base Base_BB_inicial
# ================================================================

cols_interesse = [
    'BB UID',    'Platform Title',    'Type',    'Deeplink',    'Seasons',    'Episodes',    'Season Numbers',    'BB Cast',    'BB Countries',    'BB Directors',    'BB Duration',    'BB Languages',
    'BB Original Title',    'BB Primary Company',    'BB Primary Country',    'BB Primary Genre',    'BB Production Companies',    'BB Title',    'BB Year',    'IMDb ID']

# Mantém somente as colunas listadas
Base_BB_trim = Base_BB_inicial[cols_interesse].copy()

# Exibe confirmação e amostra
display(HTML(f"<h4 style='color:#2a5599;'>📋 Base_BB_trim criada com {len(Base_BB_trim):,} registros e {len(Base_BB_trim.columns)} colunas.</h4>"))
display(Base_BB_trim.sample(5).style.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
    {'selector': 'td', 'props': [('padding', '6px 12px')]}
]))


In [ ]:
Base_BB_trim.info()

In [ ]:
# Exportando o dataframe Base_BB para o formato Excel
Base_BB_trim.to_excel('Bases/Base_BB_inicial_cortada.xlsx', index=False, engine='openpyxl')


In [ ]:
# Recuperando o dataframe Base_BB para economizar tempo de carregamento

Base_BB = pd.read_excel('Bases/Base_BB_inicial_cortada.xlsx', engine='openpyxl')

### 2.2) Preenchimento de vazios SAD

In [ ]:
# ================================================================
# 2.2) Preenchimento e ajustes de colunas na Base_SAD
# ================================================================

# 1️⃣ Preencher vazios nos campos textuais com NaN explícito
Base_SAD['Título Original'] = Base_SAD['Título Original'].fillna(np.nan)
Base_SAD['Produtora'] = Base_SAD['Produtora'].fillna(np.nan)
Base_SAD['Diretor'] = Base_SAD['Diretor'].fillna(np.nan)

# 2️⃣ Renomear coluna 'Duração' para 'Duração Obra'
if 'Duração' in Base_SAD.columns:
    Base_SAD = Base_SAD.rename(columns={'Duração': 'Duração Obra'})

# 3️⃣ Preencher vazios em 'Duração Obra' com zero
if 'Duração Obra' in Base_SAD.columns:
    Base_SAD['Duração Obra'] = Base_SAD['Duração Obra'].fillna(0)

# 4️⃣ Renomear coluna 'ID_OBRA' para 'Nº CPB'
if 'ID_OBRA' in Base_SAD.columns:
    Base_SAD = Base_SAD.rename(columns={'ID_OBRA': 'Nº CPB'})

# 5️⃣ Mensagem de confirmação
display(HTML(
    "<h4 style='color:#2a5599;'>✅ Preenchimento e renomeações concluídos com sucesso para <code>Base_SAD</code>.</h4>"
))

display(Base_SAD.sample(10).style.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
    {'selector': 'td', 'props': [('padding', '6px 12px')]}
]))

### 2.3) Preenchimento de vazios BB

In [ ]:
# ================================================================
# 2.3) Preenchimento de vazios na Base_BB
# ================================================================

# 1️⃣ Campos textuais – preencher com string vazia ''
campos_texto = [
    'Platform Title', 'Type', 'Deeplink', 'Season Numbers',
    'BB Cast', 'BB Countries', 'BB Directors', 'BB Languages',
    'BB Original Title', 'BB Primary Company', 'BB Primary Country',
    'BB Primary Genre', 'BB Production Companies', 'BB Title', 'IMDb ID'
]

for col in campos_texto:
    if col in Base_BB.columns:
        Base_BB[col] = Base_BB[col].fillna('').astype(str)

# 2️⃣ Campos numéricos – preencher com 0 e converter para inteiro
campos_inteiros = ['Seasons', 'Episodes', 'BB Duration', 'BB Year']

for col in campos_inteiros:
    if col in Base_BB.columns:
        Base_BB[col] = Base_BB[col].fillna(0).astype(int)

# 3️⃣ Campos de identificação – garantir coerência de tipo string
if 'BB UID' in Base_BB.columns:
    Base_BB['BB UID'] = Base_BB['BB UID'].astype(str)

# 4️⃣ Confirmação visual
display(HTML(
    "<h4 style='color:#2a5599;'>✅ Preenchimento e tipagem concluídos com sucesso para <code>Base_BB</code>.</h4>"
))
display(Base_BB.sample(10).style.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
    {'selector': 'td', 'props': [('padding', '6px 12px')]}
]))


In [ ]:
Base_BB.sample(10)

### 2.3) Criação dos dataframes e verificação dos resultados

In [ ]:
# Separando a Base_SAD em filmes e séries
filmesBR_SAD = Base_SAD[Base_SAD["Organização Temporal"] == "NÃO SERIADA"]
seriesBR_SAD = Base_SAD[Base_SAD["Organização Temporal"] != "NÃO SERIADA"]


In [ ]:
# Separando a Base_BB em filmes e séries, considerando também a origem BR ou Estrangeira
filmesBR_BB = Base_BB[(Base_BB['BB Countries'].str.contains("BR")) & (Base_BB['Type'].str.contains("Movie"))]
seriesBR_BB = Base_BB[(Base_BB['BB Countries'].str.contains("BR")) & (Base_BB['Type'].str.contains("Series"))]
filmesEstr_BB = Base_BB[(~Base_BB['BB Countries'].str.contains("BR")) & (Base_BB['Type'].str.contains("Movie"))]
seriesEstr_BB = Base_BB[(~Base_BB['BB Countries'].str.contains("BR")) & (Base_BB['Type'].str.contains("Series"))]

In [ ]:
# Imprimindo a quantidade de linhas nos datasets originais
print(f"Quantidade de linhas em Base_SAD: {len(Base_SAD)}")
print(f"Quantidade de linhas em Base_BB: {len(Base_BB)}\n")

# Imprimindo a quantidade de linhas em cada dataframe derivado
print(f"Quantidade de linhas em filmesBR_SAD: {len(filmesBR_SAD)}")
print(f"Quantidade de linhas em seriesBR_SAD: {len(seriesBR_SAD)}")
print(f"Quantidade de linhas em filmesBR_BB: {len(filmesBR_BB)}")
print(f"Quantidade de linhas em seriesBR_BB: {len(seriesBR_BB)}")
print(f"Quantidade de linhas em filmesEstr_BB: {len(filmesEstr_BB)}")
print(f"Quantidade de linhas em seriesEstr_BB: {len(seriesEstr_BB)}")

# Comparação entre o input e o output
total_linhas_SAD = len(filmesBR_SAD) + len(seriesBR_SAD)
total_linhas_BB = len(filmesBR_BB) + len(seriesBR_BB) + len(filmesEstr_BB) + len(seriesEstr_BB)

print(f"\nComparação Base_SAD:")
print(f"Total de linhas original: {len(Base_SAD)}")
print(f"Total de linhas após divisão: {total_linhas_SAD}")
print(f"Diferença de linhas Base_SAD: {len(Base_SAD) - total_linhas_SAD}")

print(f"\nComparação Base_BB:")
print(f"Total de linhas original: {len(Base_BB)}")
print(f"Total de linhas após divisão: {total_linhas_BB}")
print(f"Diferença de linhas Base_BB: {len(Base_BB) - total_linhas_BB}")


### 2.4) Verificação de títulos homônimos

#### Esta verificação não está criando nem separando nenhum registro ou dataframe e serve 
#### para simples conferência podendo ser desconsiderada a dependeder do contexto

In [ ]:
Base_SAD.sample(10)

In [ ]:
# ================================================================
# 3.1) Verificação de títulos homônimos nas bases SAD e BB
# ================================================================

def verificar_homonimos(df, nome_df, col_titulo, col_ano, col_diretor, col_id, col_titulo_alt=None):
    """
    Identifica títulos homônimos dentro de uma base,
    considerando título (e opcionalmente título alternativo), ano, diretor e identificadores únicos.
    """
    # Define as colunas de agrupamento dinamicamente
    colunas_grp = [col_titulo, col_ano, col_diretor]
    if col_titulo_alt and col_titulo_alt in df.columns and col_titulo_alt != col_titulo:
        colunas_grp.insert(1, col_titulo_alt)  # insere o alternativo logo após o título principal

    # Agrupamento e contagem de identificadores distintos
    repeticoes = (
        df.groupby(colunas_grp)[col_id]
        .nunique()
        .reset_index(name='Qtd_IDs')
    )

    # Filtra apenas casos com mais de um identificador
    repeticoes = repeticoes[repeticoes['Qtd_IDs'] > 1]

    # Exibição formatada
    display(HTML(f"<h4 style='color:#2a5599;'>🎬 Verificação de homônimos — {nome_df}</h4>"))
    if repeticoes.empty:
        display(HTML("<p style='color:green;'>✅ Nenhum título homônimo encontrado.</p><hr>"))
    else:
        display(repeticoes.style.set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
            {'selector': 'td', 'props': [('padding', '6px 12px')]}
        ]))
        print(f"\nQuantidade de títulos homônimos em {nome_df}: {len(repeticoes)}\n")
        display(HTML("<hr style='margin:25px 0;'>"))


# ================================================================
# Aplicação às bases
# ================================================================

# --- Filmes e séries brasileiras (SAD)
verificar_homonimos(
    filmesBR_SAD,
    "Filmes Brasileiros (SAD)",
    "Título Original",
    "Ano de Produção",
    "Diretor",
    "Nº CPB"
)

verificar_homonimos(
    seriesBR_SAD,
    "Séries Brasileiras (SAD)",
    "Título Original",
    "Ano de Produção",
    "Diretor",
    "Nº CPB"
)

# --- Filmes e séries brasileiras (BB)
verificar_homonimos(
    filmesBR_BB,
    "Filmes Brasileiros (BB)",
    "Platform Title",
    "BB Year",
    "BB Directors",
    "BB UID",
    col_titulo_alt="BB Original Title"
)

verificar_homonimos(
    seriesBR_BB,
    "Séries Brasileiras (BB)",
    "Platform Title",
    "BB Year",
    "BB Directors",
    "BB UID",
    col_titulo_alt="BB Original Title"
)

# --- Filmes e séries estrangeiras (BB)
verificar_homonimos(
    filmesEstr_BB,
    "Filmes Estrangeiros (BB)",
    "Platform Title",
    "BB Year",
    "BB Directors",
    "BB UID",
    col_titulo_alt="BB Original Title"
)

verificar_homonimos(
    seriesEstr_BB,
    "Séries Estrangeiras (BB)",
    "Platform Title",
    "BB Year",
    "BB Directors",
    "BB UID",
    col_titulo_alt="BB Original Title"
)


## 3) Processamento para uniformização de campos de texto e numéricos

### 3.1) Definição das funções de processamento

In [ ]:
# Criação de cópias limpas dos DataFrames
filmesBR_SAD_limpo = filmesBR_SAD.copy()
seriesBR_SAD_limpo = seriesBR_SAD.copy()

filmesBR_BB_limpo = filmesBR_BB.copy()
seriesBR_BB_limpo = seriesBR_BB.copy()

filmesEstr_BB_limpo = filmesEstr_BB.copy()
seriesEstr_BB_limpo = seriesEstr_BB.copy()


In [ ]:
# Função para transformar numerais romanos para arábicos
def roman_to_arabic(roman):
    roman_numerals = {
        'i': 1,
        'ii': 2,
        'iii': 3,
        'iv': 4,
        'v': 5,
        'vi': 6,
        'vii': 7,
        'viii': 8,
        'ix': 9,
        'x': 10
    }
    
    return roman_numerals.get(roman.lower(), None)

In [ ]:
def _colapsar_sigla_pontilhada(s):
    if not isinstance(s, str):
        return s
    # casa sequências de letra+ponto repetidas (2+ letras), opcional letra final e ponto
    # exemplos cobertos: "L.A.P.A.", "U.S.A", "F.B.I.", "L. A. P. A."
    return re.sub(r'(?i)(?:\b[a-z]\s*\.){2,}[a-z]?\s*\.?', 
                  lambda m: re.sub(r'[\s\.]', '', m.group(0)), 
                  s)

In [ ]:
def nonempty_str(series_or_str):
    s = pd.Series(series_or_str) if not isinstance(series_or_str, pd.Series) else series_or_str
    s = s.astype(str).str.strip()
    return s.notna() & s.ne('') & s.str.lower().ne('nan') & s.str.lower().ne('none')

In [ ]:
# Função para realizar as transformações de strings
def process_string3(s):
    if not isinstance(s, str):  # Se não for uma string, retorne como está
        return s
    
    # Corrigir espaçamento antes e depois de vírgulas
    s = re.sub(r'\s*,\s*', ', ', s)
    s = s.lower().strip()  # Converter texto para minúsculo e remover espaços no começo/fim
    
    # Remover caracteres não latinos (Chineses por exemplo)
    s = re.sub(r'[^\u0000-\u024F\u1E00-\u1EFF]+', '', s)
    
    # Remover referências de temporadas
    patterns = [
    r'(\s*-?\s*season\s*\d+)$',
    r'(\s*-?\s*\d+\s*[ºª°]?\s*temporada)$',
    r'(\s*-?\s*\d+[aª]?\s*temp)$',
    r'(\s*-?\s*\d+rd\s*season)$',
    r'(\s*-?\s*\d+th\s*season)$',
    r'(\s*-?\s*(primeira|segunda|terceira|quarta|quinta|sexta|sétima|oitava|nona|décima)\s*temporada)$',
    ]
    for pattern in patterns:
        s = re.sub(pattern, '', s, flags=re.IGNORECASE)
    
    
    # Converter numerais romanos para arábicos
    words = s.split()
    for i, word in enumerate(words):
        if word in ['i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x']:
            words[i] = str(roman_to_arabic(word))
    s = ' '.join(words)
    
    # Substituir ° por o antes de remover acentos
    s = s.replace('°', 'o')
    s = s.replace('°', 'o')
    s = s.replace('ª', 'a')

    # COLAPSAR SIGLAS PONTILHADAS ANTES DE REMOVER PONTUAÇÃO
    s = _colapsar_sigla_pontilhada(s)
    
    # Transformar caracteres de pontuação em espaços
    for char in string.punctuation:
        s = s.replace(char, ' ')
    
    # Converter caracteres acentuados para sua versão sem acentuação
    s = unidecode.unidecode(s)
    
    # Remover palavras comuns
    common_words = ['o', 'a', 'os', 'as', 'um', 'uma', 'do', 'da', 'dos', 'das', 'e', 'the']
    s = ' '.join([word for word in s.split() if word not in common_words])
    
    # Remover espaços excessivos
    s = re.sub(' +', ' ', s)   
    

    
    return s




In [ ]:
def remove_palavras_específicas2(s):
    """
    Remove múltiplas siglas empresariais no final do nome da produtora.
    Exemplos:
      - 'LTDA - ME'
      - 'EIRELI - ME'
      - 'LTDA. - ME - BAIXADO'
    """
    if not isinstance(s, str):
        return s

    s = " " + s.strip() + " "

    # Padroniza pontuação comum
    s = re.sub(r'[\.\-–]+', ' ', s)

    # Lista ampliada de siglas empresariais comuns
    palavras_especificas = [
        's', 'me', 'ltd', 'ltda', 'inc', 'sa', 'eireli', 'mei', 'corp', 'epp', 'ei'
    ]

    # Remove todas as ocorrências dessas siglas no final (pode haver mais de uma)
    while True:
        s_antigo = s
        for palavra in palavras_especificas:
            s = re.sub(rf'\s{palavra}\s*$', ' ', s, flags=re.IGNORECASE)
        if s == s_antigo:
            break

    return s.strip()

In [ ]:
#  Remover CPFs ou blocos numéricos no início

def remove_cpf_inicio(s):
    """
    Remove CPF ou blocos numéricos no início da string.
    Casos tratados:
      - '123456789 Nome'
      - '123.456.789 Nome'
    """
    if not isinstance(s, str):
        return s

    s = s.strip()

    # Caso 1: primeiros 9 caracteres são todos dígitos + espaço
    if re.match(r'^\d{9}\s', s):
        return s[10:].strip()

    # Caso 2: padrão CPF formatado com pontos — 11 dígitos no formato XXX.XXX.XXX
    s = re.sub(r'^\d{3}\.\d{3}\.\d{3}\s+', '', s)

    return s.strip()


In [ ]:
#  Remover CPFs ou blocos numéricos no final

def remove_cpf_fim(s):
    """
    Remove CPF no FINAL do texto da produtora.
    Casos tratados:
      - '... 123456789'
      - '... 123.456.789'
      - '... - 123456789' / '... - 123.456.789'
      - '... (123456789)' / '... (123.456.789)'
    Não altera outros números, pois exige exatamente 9 dígitos (com ou sem pontos).
    """
    if not isinstance(s, str):
        return s

    s = s.strip()

    # remove bloco final: [opcional ' - ' ou parênteses] + (XXX.XXX.XXX | XXXXXXXXX) + final de string
    s = re.sub(r'[\s\-\–\(\)]*(?:\d{3}\.\d{3}\.\d{3}|\d{9})\s*$', '', s, flags=re.IGNORECASE)

    return s.strip()


In [ ]:
# Função para transformar o campo de texto do ano de produção em dois campos numericos: Ano inicial e Ano final
def extract_ano_inicial(year_value):
    # Converter o valor para string
    year_string = str(year_value)
    
    # Se 'A' estiver presente, extraia o ano antes do 'A'
    if 'A' in year_string:
        return int(year_string.split(' A ')[0])
    # Se for apenas um ano, retorne esse ano
    elif year_string.isdigit():
        return int(year_string)
    else:
        return None

def extract_ano_final(year_value):
    # Converter o valor para string
    year_string = str(year_value)
    
    # Se 'A' estiver presente, extraia o ano após o 'A'
    if 'A' in year_string:
        return int(year_string.split(' A ')[1])
    # Se for apenas um ano, retorne esse mesmo ano
    elif year_string.isdigit():
        return int(year_string)
    else:
        return None

In [ ]:
def remove_last_baixada(s):
    """
    Remove ocorrências de 'baixado' ou 'baixada' (com hífen, espaço ou parênteses).
    Exemplos tratados:
      - '(BAIXADA)'
      - '- BAIXADO'
      - '- BAIXADA'
      - '(BAIXADO)'
    """
    if not isinstance(s, str):
        return s

    s = s.strip()

    # Remove qualquer ocorrência no final, com ou sem parênteses ou hífen
    pattern = r'[\s\-\(]*(baixad[ao])[\s\)]*$'
    return re.sub(pattern, '', s, flags=re.IGNORECASE).strip()


In [ ]:
particles_pt = {'da','de','do','das','dos'}

def _normalize_name_basic(s: str) -> str:
    s = '' if pd.isna(s) else str(s).strip().lower()
    s = unidecode.unidecode(s)
    s = re.sub(r'[^\w\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def surnames_from_raw(raw: str) -> set:
    """
    Divide por VÍRGULA, normaliza cada nome e extrai o sobrenome 'de verdade':
    - último token; se penúltimo ∈ {da,de,do,das,dos}, pega 'penúltimo + último' (ex.: 'da silva').
    """
    if pd.isna(raw) or not str(raw).strip():
        return set()
    out = set()
    for p in str(raw).split(','):
        n = _normalize_name_basic(p)
        if not n:
            continue
        toks = n.split()
        if not toks:
            continue
        # '... da silva' -> 'da silva'
        if len(toks) >= 2 and toks[-2] in particles_pt:
            out.add(toks[-2] + ' ' + toks[-1])
        else:
            out.add(toks[-1])
    return out

def tokens_empresas(s: str) -> set:
    """
    Usa campos *_processado (já sem ltda, me, etc.) e quebra em tokens distintivos.
    """
    if pd.isna(s) or not str(s).strip():
        return set()
    t = str(s).strip().split()
    return set([w for w in t if len(w) > 2])  # ignora tokens 1–2 letras

### 3.2) Aplicação das Funções para uniformização de campos textuais

In [ ]:
# ===================================================================================
# 3.2 Aplicação das Funções para uniformização de campos textuais
# ===================================================================================

# ================================================
# Bases SAD
# ================================================
for df in [filmesBR_SAD_limpo, seriesBR_SAD_limpo]:
    
    # --- Normalização base (acentos, pontuação, caixa etc.)
    df['Título Original_processado'] = df['Título Original'].apply(process_string3)
    df['Diretor_processado']         = df['Diretor'].apply(process_string3)
    df['Produtora_processada']       = df['Produtora'].apply(process_string3)
    
    # --- Remoção de CPF no início e no final das strings
    df['Produtora_processada'] = df['Produtora_processada'].apply(remove_cpf_inicio)
    df['Produtora_processada'] = df['Produtora_processada'].apply(remove_cpf_fim)
    
    # --- Remoção de indicações de "baixado(a)" em diferentes formatos
    df['Produtora_processada'] = df['Produtora_processada'].apply(remove_last_baixada)
    df['Diretor_processado']   = df['Diretor_processado'].apply(remove_last_baixada)
    
    # --- Remoção de siglas empresariais (LTDA, ME, EIRELI etc.)
    df['Produtora_processada'] = df['Produtora_processada'].apply(remove_palavras_específicas2)


# ================================================
# Bases BB_BR
# ================================================
for df in [filmesBR_BB_limpo, seriesBR_BB_limpo]:
    
    # --- Normalização base
    df['Platform Title_processado']          = df['Platform Title'].apply(process_string3)
    df['BB Original Title_processado']       = df['BB Original Title'].apply(process_string3)
    df['BB Primary Company_processado']      = df['BB Primary Company'].apply(process_string3)
    df['BB Production Companies_processado'] = df['BB Production Companies'].apply(process_string3)
    df['BB Directors_processado']            = df['BB Directors'].apply(process_string3)
    df['BB Title_processado']                = df['BB Title'].apply(process_string3)
    
    # --- Remoção de CPF no início e no final das strings (para produtoras)
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_cpf_inicio)
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_cpf_fim)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_cpf_inicio)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_cpf_fim)
    
    # --- Remoção de "baixado(a)" e variantes
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_last_baixada)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_last_baixada)
    
    # --- Remoção de siglas empresariais (LTDA, ME, EIRELI etc.)
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_palavras_específicas2)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_palavras_específicas2)


# ================================================
# Bases BB_estrangeiras
# ================================================

for df in [filmesEstr_BB_limpo, seriesEstr_BB_limpo]:
    # 1) Normalização base
    df['Platform Title_processado']          = df['Platform Title'].apply(process_string3)
    df['BB Original Title_processado']       = df['BB Original Title'].apply(process_string3)
    df['BB Primary Company_processado']      = df['BB Primary Company'].apply(process_string3)
    df['BB Production Companies_processado'] = df['BB Production Companies'].apply(process_string3)
    df['BB Directors_processado']            = df['BB Directors'].apply(process_string3)
    df['BB Title_processado']                = df['BB Title'].apply(process_string3)

    # 2) Remoção de CPF no início e no fim (produtoras)
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_cpf_inicio).apply(remove_cpf_fim)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_cpf_inicio).apply(remove_cpf_fim)

    # 3) Remoção de “baixado/baixada”
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_last_baixada)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_last_baixada)

    # 4) Remoção de siglas empresariais
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_palavras_específicas2)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_palavras_específicas2)



In [ ]:
# ===================================================================================
# 3.3 Amostras das bases processadas
# ===================================================================================

def mostrar_amostra(df, titulo):
    display(HTML(f"<h4 style='color:#006699;'>{titulo}</h4>"))
    display(df.sample(5).style.set_table_attributes("style='display:inline'").set_caption(titulo))
    display(HTML("<hr style='margin:25px 0;'>"))

# Exibir amostras
mostrar_amostra(filmesBR_SAD_limpo, "🎬 Filmes Brasileiros (SAD)")
mostrar_amostra(seriesBR_SAD_limpo, "📺 Séries Brasileiras (SAD)")
mostrar_amostra(filmesBR_BB_limpo, "🎞️ Filmes Brasileiros (BB)")
mostrar_amostra(seriesBR_BB_limpo, "📡 Séries Brasileiras (BB)")
mostrar_amostra(filmesEstr_BB_limpo, "🌍 Filmes Estrangeiros (BB)")
mostrar_amostra(seriesEstr_BB_limpo, "🌍 Séries Estrangeiras (BB)")



### 3.3) Criação e tratamento dos campos de Ano de Produção

In [ ]:
# Desativar warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Aplicando as funções e criando as novas colunas nos DataFrames SAD
for df in [filmesBR_SAD_limpo, seriesBR_SAD_limpo]:
    df['Ano Inicial'] = df['Ano de Produção'].apply(extract_ano_inicial)
    df['Ano Final'] = df['Ano de Produção'].apply(extract_ano_final)

In [ ]:
for df in [filmesBR_BB_limpo, seriesBR_BB_limpo]:
    df['BB Year'] = df['BB Year'].fillna(0).astype(int)
    
# Reativar warnings
warnings.simplefilter(action='default', category=pd.errors.SettingWithCopyWarning)


In [ ]:
# A) Conversão de tipos numéricos (SAD) com validação
cols_int_sad = ['Ano Inicial',	'Ano Final', 'Nº de Episódios'] 

def to_nullable_int(series, nome):
    s = pd.to_numeric(series, errors='coerce')
    frac = (s % 1).fillna(0)
    # valida: só aceita inteiros exatos (fração = 0)
    if (frac != 0).any():
        # mostra exemplos problemáticos e aborta
        bad = series[(frac != 0)]
        raise ValueError(f"Coluna '{nome}' contém valores não-inteiros, ex.: {bad.head(10).tolist()}")
    return s.astype('Int64')

for c in cols_int_sad:
    if c in filmesBR_SAD_limpo.columns:
        filmesBR_SAD_limpo[c] = to_nullable_int(filmesBR_SAD_limpo[c], c)
    if c in seriesBR_SAD_limpo.columns:
        seriesBR_SAD_limpo[c]  = to_nullable_int(seriesBR_SAD_limpo[c], c)


In [ ]:
# ================================================================
# 3.3 — Materialização de features para Matching (pré-cálculo)
# ================================================================

def _prod_tokens_union(row):
    s1 = row['BB Primary Company_processado']
    s2 = row['BB Production Companies_processado']
    toks = set()
    if isinstance(s1, str) and s1.strip():
        toks |= tokens_empresas(s1)
    if isinstance(s2, str) and s2.strip() and s2 != s1:
        toks |= tokens_empresas(s2)
    return toks

# SAD (filmes e séries)
for df in [filmesBR_SAD_limpo, seriesBR_SAD_limpo]:
    df['dir_set_sad']      = df['Diretor'].apply(surnames_from_raw)             # usa campo RAW (separado por vírgula)
    df['prod_tokens_sad']  = df['Produtora_processada'].apply(tokens_empresas)  # já normalizada

# BB (filmes/séries nacionais e estrangeiros)
for df in [filmesBR_BB_limpo, seriesBR_BB_limpo, filmesEstr_BB_limpo, seriesEstr_BB_limpo]:
    df['dir_set_bb']       = df['BB Directors'].apply(surnames_from_raw)        # usa campo RAW (separado por vírgula)
    df['prod_tokens_bb']   = df.apply(_prod_tokens_union, axis=1)               # união: Primary + Companies (se diferentes)


### 3.5) Verificação dos resultados após transformações

In [ ]:
print(filmesBR_SAD_limpo.info())  # Resumo dos atributos
print("\nResumo Estatístico:")
print(filmesBR_SAD_limpo.describe())  # Resumo estatístico

In [ ]:
print(seriesBR_SAD_limpo.info())  # Resumo dos atributos
print("\nResumo Estatístico:")
print(seriesBR_SAD_limpo.describe())  # Resumo estatístico

In [ ]:
print(filmesBR_BB_limpo.info())  # Resumo dos atributos
print("\nResumo Estatístico:")
print(filmesBR_BB_limpo.describe())  # Resumo estatístico

In [ ]:
print(seriesBR_BB_limpo.info())  # Resumo dos atributos
print("\nResumo Estatístico:")
print(seriesBR_BB_limpo.describe())  # Resumo estatístico

In [ ]:
print(filmesEstr_BB_limpo.info())  # Resumo dos atributos
print("\nResumo Estatístico:")
print(filmesEstr_BB_limpo.describe())  # Resumo estatístico


In [ ]:
print(seriesEstr_BB_limpo.info())  # Resumo dos atributos
print("\nResumo Estatístico:")
print(seriesEstr_BB_limpo.describe())  # Resumo estatístico


### 3.6) Salva bases Limpas para importação futura

In [ ]:
# Nome do arquivo de saída
arquivo = 'Pre-processamento bases limpas.xlsx'

# Usando ExcelWriter para salvar os DataFrames em abas diferentes
with pd.ExcelWriter(arquivo, engine='openpyxl') as writer:
    # Escrevendo cada DataFrame em uma aba separada
    filmesBR_SAD_limpo.to_excel(writer, sheet_name='Filmes SAD', index=False)
    seriesBR_SAD_limpo.to_excel(writer, sheet_name='Séries SAD', index=False)
    filmesBR_BB_limpo.to_excel(writer, sheet_name='Filmes BR BB', index=False)
    seriesBR_BB_limpo.to_excel(writer, sheet_name='Séries BR BB', index=False)
    filmesEstr_BB_limpo.to_excel(writer, sheet_name='Filmes Estr BB', index=False)
    seriesEstr_BB_limpo.to_excel(writer, sheet_name='Séries Estr BB', index=False)


In [ ]:
# Recupera bases limpas
arquivo = 'Pre-processamento bases limpas.xlsx'
filmesBR_SAD_limpo = pd.read_excel(arquivo, sheet_name='Filmes SAD')
seriesBR_SAD_limpo = pd.read_excel(arquivo, sheet_name='Séries SAD')
filmesBR_BB_limpo = pd.read_excel(arquivo, sheet_name='Filmes BR BB')
seriesBR_BB_limpo = pd.read_excel(arquivo, sheet_name='Séries BR BB')
filmesEstr_BB_limpo = pd.read_excel(arquivo, sheet_name='Filmes Estr BB')
seriesEstr_BB_limpo = pd.read_excel(arquivo, sheet_name='Séries Estr BB')

In [ ]:
def _fix_headers(df):
    df.columns = (df.columns.astype(str)
                  .str.replace('\u00A0', ' ', regex=False)   # NBSP
                  .str.replace('\ufeff', '', regex=False)    # BOM
                  .str.replace(r'\s+', ' ', regex=True)      # espaços duplicados
                  .str.strip())
    return df

for df in [filmesBR_BB_limpo, seriesBR_BB_limpo, filmesEstr_BB_limpo, seriesEstr_BB_limpo,
           filmesBR_SAD_limpo, seriesBR_SAD_limpo]:
    _fix_headers(df)


In [ ]:
required_bb = [
    'BB UID','BB Year','BB Directors','BB Primary Company','BB Production Companies',
    'BB Title_processado','BB Original Title_processado','Platform Title_processado'
]

for name, d in {
    'filmesBR_BB_limpo': filmesBR_BB_limpo,
    'seriesBR_BB_limpo': seriesBR_BB_limpo,
    'filmesEstr_BB_limpo': filmesEstr_BB_limpo,
    'seriesEstr_BB_limpo': seriesEstr_BB_limpo,
}.items():
    missing = [c for c in required_bb if c not in d.columns]
    if missing:
        print(f"{name} faltando:", missing)


## 4) Busca de Correspondências exatas de Título, Diretor e Ano

### 4.1) Definição de funções de busca

In [ ]:
# ================================================================
# 4.1 Funções — motor de match limpo (título =, ano ±2, diretor obrigatório)
# ================================================================

ANO_TOL = 2  # tolerância

def _cpb_col(df):
    return 'Nº CPB' if 'Nº CPB' in df.columns else ('N° CPB' if 'N° CPB' in df.columns else 'Nº CPB')

def build_bb_title_keys(df_bb):
    title_cols = [
        'BB Title_processado',
        'BB Original Title_processado',
        'Platform Title_processado'
    ]
    keep = [
        'BB UID','BB Year','BB Directors',
        'BB Primary Company','BB Production Companies',
        # >>> campos processados que faltavam <<<
        'BB Primary Company_processado','BB Production Companies_processado',
        'BB Title','BB Original Title','Platform Title'
    ]
    m = df_bb[keep + title_cols].melt(
        id_vars=keep,
        value_vars=title_cols,
        var_name='title_source',
        value_name='BB_title_key'
    )
    m = m[m['BB_title_key'].notna() & (m['BB_title_key'] != '')]
    m = m.drop_duplicates(['BB UID','BB_title_key'], keep='first')
    return m


def match_pair_min(df_sad, df_bb, categoria):
    cpb = _cpb_col(df_sad)

    # 1) título
    bbk = build_bb_title_keys(df_bb)
    sadk = df_sad[[cpb, 'Título Original','Título Original_processado',
                   'Diretor','Ano Inicial','Ano Final','Produtora']].copy()
    cand = bbk.merge(sadk, left_on='BB_title_key', right_on='Título Original_processado', how='inner')

    # 2) ano ±2
    cand = cand.dropna(subset=['BB Year','Ano Inicial']).copy()
    cand['Ano Final'] = cand['Ano Final'].fillna(cand['Ano Inicial'])
    mask_ano = (
        (cand['BB Year'].astype(int) >= (cand['Ano Inicial'].astype(int) - ANO_TOL)) &
        (cand['BB Year'].astype(int) <= (cand['Ano Final'].astype(int)   + ANO_TOL))
    )
    cand = cand[mask_ano]

    # 3) diretor obrigatório
    a = cand['BB Directors'].apply(surnames_from_raw)
    b = cand['Diretor'].apply(surnames_from_raw)
    cand['dir_overlap'] = [sorted(list(x & y)) for x, y in zip(a, b)]
    cand = cand[cand['dir_overlap'].map(len) > 0].copy()

    # 4) saída
    out = cand[[cpb, 'BB UID',
                'Título Original',
                'BB Title','BB Original Title','Platform Title',
                'Diretor','BB Directors',
                'Ano Inicial','Ano Final','BB Year',
                'Produtora','BB Primary Company','BB Production Companies']].copy()

    out['categoria']  = categoria
    out['match_rule'] = 'titulo+ano+diretor'
    out = out.rename(columns={'BB UID':'UID', 'BB Title':'Title',
                              'BB Directors':'Directors', 'BB Year':'Year',
                              cpb: cpb})
    return out


In [ ]:
# ================================================================
# 4.1b — regra 2: título =, ano ±2, PRODUTORA (tokens) obrigatória
# ================================================================

def _bb_prod_tokens_row(r):
    s1 = str(r.get('BB Primary Company_processado', '') or '').strip()
    s2 = str(r.get('BB Production Companies_processado', '') or '').strip()
    toks = set()
    if s1:
        toks |= tokens_empresas(s1)
    if s2 and s2 != s1:
        toks |= tokens_empresas(s2)
    return toks

def match_pair_produtora(df_sad, df_bb, categoria):
    cpb = _cpb_col(df_sad)

    # 1) título
    bbk = build_bb_title_keys(df_bb)
    sadk = df_sad[[cpb, 'Título Original','Título Original_processado',
                   'Produtora','Produtora_processada',
                   'Ano Inicial','Ano Final',
                   'Diretor']].copy()  # diretor só para visualização/auditoria
    cand = bbk.merge(sadk, left_on='BB_title_key', right_on='Título Original_processado', how='inner')

    # 2) ano ±2
    cand = cand.dropna(subset=['BB Year','Ano Inicial']).copy()
    cand['Ano Final'] = cand['Ano Final'].fillna(cand['Ano Inicial'])
    mask_ano = (
        (cand['BB Year'].astype(int) >= (cand['Ano Inicial'].astype(int) - ANO_TOL)) &
        (cand['BB Year'].astype(int) <= (cand['Ano Final'].astype(int)   + ANO_TOL))
    )
    cand = cand[mask_ano]

    # 3) PRODUTORA — overlap de tokens
    cand['prod_bb']  = cand.apply(_bb_prod_tokens_row, axis=1)
    cand['prod_sad'] = cand['Produtora_processada'].apply(tokens_empresas)
    cand['prod_overlap'] = [sorted(list(x & y)) for x, y in zip(cand['prod_bb'], cand['prod_sad'])]
    cand = cand[cand['prod_overlap'].map(len) > 0].copy()

    # 4) saída
    out = cand[[cpb, 'BB UID',
                'Título Original',
                'BB Title','BB Original Title','Platform Title',
                'Diretor','BB Directors',
                'Ano Inicial','Ano Final','BB Year',
                'Produtora','BB Primary Company','BB Production Companies']].copy()

    out['categoria']  = categoria
    out['match_rule'] = 'titulo+ano+produtora'
    out = out.rename(columns={'BB UID':'UID', 'BB Title':'Title',
                              'BB Directors':'Directors', 'BB Year':'Year',
                              cpb: cpb})
    return out


In [ ]:
# ================================================================
# 4.2 — Executar PRODUTORA e unir com DIRETOR (prioridade ao DIRETOR)
# ================================================================

# Se você já rodou a 1ª rodada e tem out_* da regra diretor, mantenha:
dir_filmes_BR   = match_pair_min(filmesBR_SAD_limpo,  filmesBR_BB_limpo,   'filmes_BR')
dir_series_BR   = match_pair_min(seriesBR_SAD_limpo,  seriesBR_BB_limpo,   'series_BR')
dir_filmes_EST  = match_pair_min(filmesBR_SAD_limpo,  filmesEstr_BB_limpo, 'filmes_EST')
dir_series_EST  = match_pair_min(seriesBR_SAD_limpo,  seriesEstr_BB_limpo, 'series_EST')

matches_dir = pd.concat([dir_filmes_BR, dir_series_BR, dir_filmes_EST, dir_series_EST],
                        ignore_index=True)

# Filtrar os BB ainda não casados para a rodada PRODUTORA (opcional, mas recomendado)
uids_casados = set(matches_dir['UID'].unique())

filmesBR_BB_rest   = filmesBR_BB_limpo[~filmesBR_BB_limpo['BB UID'].isin(uids_casados)]
seriesBR_BB_rest   = seriesBR_BB_limpo[~seriesBR_BB_limpo['BB UID'].isin(uids_casados)]
filmesEstr_BB_rest = filmesEstr_BB_limpo[~filmesEstr_BB_limpo['BB UID'].isin(uids_casados)]
seriesEstr_BB_rest = seriesEstr_BB_limpo[~seriesEstr_BB_limpo['BB UID'].isin(uids_casados)]

prod_filmes_BR   = match_pair_produtora(filmesBR_SAD_limpo,  filmesBR_BB_rest,   'filmes_BR')
prod_series_BR   = match_pair_produtora(seriesBR_SAD_limpo,  seriesBR_BB_rest,   'series_BR')
prod_filmes_EST  = match_pair_produtora(filmesBR_SAD_limpo,  filmesEstr_BB_rest, 'filmes_EST')
prod_series_EST  = match_pair_produtora(seriesBR_SAD_limpo,  seriesEstr_BB_rest, 'series_EST')

matches_prod = pd.concat([prod_filmes_BR, prod_series_BR, prod_filmes_EST, prod_series_EST],
                         ignore_index=True)

# União final: DIRETOR tem prioridade; depois PRODUTORA completa o que faltou
# (se preferir, pode trocar 'keep="first"' por 'keep="last"' invertendo a prioridade)
cpb_col = 'Nº CPB' if 'Nº CPB' in filmesBR_SAD_limpo.columns else 'N° CPB'
matches_all = pd.concat([matches_dir, matches_prod], ignore_index=True)
matches_all = matches_all.drop_duplicates(subset=['UID', cpb_col], keep='first')

# Contagens
print("Contagens por categoria (após união):")
print(matches_all.groupby(['categoria','match_rule']).size().unstack(fill_value=0))
print("\nUIDs únicos casados (total):", matches_all['UID'].nunique())
print("CPBs únicos casados (total):", matches_all[cpb_col].nunique())

# Amostra para inspeção
display(matches_all.head(12))


In [ ]:
# ================================================================
# 4.3 Resultados — contagens e amostra
# ================================================================

print("Pares por categoria:")
print(matches_all['categoria'].value_counts(), "\n")

print("UIDs únicos casados:", matches_all['UID'].nunique())
print("CPBs únicos casados:", matches_all['Nº CPB'].nunique(), "\n")  # ajuste p/ 'N° CPB' se for o seu caso

display(matches_all.head(12))


In [ ]:
# ================================================================
# Sumarização por categoria com PRODUTORA (+ ganhos novos)
# Requer: matches_dir, matches_prod, build_bb_title_keys, surnames_from_raw,
#         tokens_empresas, _bb_prod_tokens_row (definidas antes)
# ================================================================

def funil_uid_com_prod(df_sad, df_bb, categoria, matches_dir, matches_prod):
    # base = UIDs únicos do BB
    obras = df_bb['BB UID'].nunique()

    # --- título (sem ano) ---
    bbk  = build_bb_title_keys(df_bb)
    sadk = df_sad[['Título Original_processado', 'Diretor',
                   'Produtora_processada', 'Ano Inicial', 'Ano Final']].copy()
    e_title = bbk.merge(sadk, left_on='BB_title_key',
                        right_on='Título Original_processado', how='inner')

    uids_titulo = set(e_title['BB UID'].unique())
    n_titulo = len(uids_titulo)

    # --- diretor (sem ano) ---
    a = e_title['BB Directors'].apply(surnames_from_raw)
    b = e_title['Diretor'].apply(surnames_from_raw)
    dir_ok = [len(x & y) > 0 for x, y in zip(a, b)]
    uids_dir = set(e_title[dir_ok]['BB UID'].unique())
    n_dir = len(uids_dir)

    # --- produtora (sem ano) ---
    prod_bb  = e_title.apply(_bb_prod_tokens_row, axis=1)
    prod_sad = e_title['Produtora_processada'].apply(tokens_empresas)
    prod_ok  = [len(x & y) > 0 for x, y in zip(prod_bb, prod_sad)]
    uids_prod = set(e_title[prod_ok]['BB UID'].unique())
    n_prod = len(uids_prod)

    # --- finais por regra (já com ano) a partir das saídas de cada regra ---
    u_dir_final  = set(matches_dir.loc[matches_dir['categoria']==categoria, 'UID'].unique())
    u_prod_final = set(matches_prod.loc[matches_prod['categoria']==categoria, 'UID'].unique())

    final_total_uids   = u_dir_final | u_prod_final
    novos_via_produtor = u_prod_final - u_dir_final

    # conversões
    conv_total_titulo   = (n_titulo / obras * 100) if obras else 0.0
    conv_tit_dir        = (n_dir    / n_titulo * 100) if n_titulo else 0.0
    conv_tit_prod       = (n_prod   / n_titulo * 100) if n_titulo else 0.0
    conv_total          = (len(final_total_uids) / obras * 100) if obras else 0.0

    return pd.DataFrame([{
        'categoria': categoria,
        'obras': obras,
        'regra_titulo': n_titulo,
        'regra_diretor': n_dir,
        'regra_produtora': n_prod,
        'final_dir_ano': len(u_dir_final),
        'final_prod_ano': len(u_prod_final),
        'final_total': len(final_total_uids),
        'novos_via_produtora': len(novos_via_produtor),
        'conv_total_titulo_%': round(conv_total_titulo, 2),
        'conv_titulo→diretor_%': round(conv_tit_dir, 2),
        'conv_titulo→produtora_%': round(conv_tit_prod, 2),
        'conv_total_%': round(conv_total, 2),
    }])

    

In [ ]:
# ---- Rodar por categoria ----
sum_f_br = funil_uid_com_prod(filmesBR_SAD_limpo,  filmesBR_BB_limpo,   'filmes_BR',  matches_dir, matches_prod)
sum_s_br = funil_uid_com_prod(seriesBR_SAD_limpo,  seriesBR_BB_limpo,   'series_BR',  matches_dir, matches_prod)
sum_f_es = funil_uid_com_prod(filmesBR_SAD_limpo,  filmesEstr_BB_limpo, 'filmes_EST', matches_dir, matches_prod)
sum_s_es = funil_uid_com_prod(seriesBR_SAD_limpo,  seriesEstr_BB_limpo, 'series_EST', matches_dir, matches_prod)

sum_cat = pd.concat([sum_f_br, sum_s_br, sum_f_es, sum_s_es], ignore_index=True)

# ---- Global (somando categorias) ----
glob = pd.DataFrame([{
    'obras':            sum_cat['obras'].sum(),
    'regra_titulo':     sum_cat['regra_titulo'].sum(),
    'regra_diretor':    sum_cat['regra_diretor'].sum(),
    'regra_produtora':  sum_cat['regra_produtora'].sum(),
    'final_dir_ano':    sum_cat['final_dir_ano'].sum(),
    'final_prod_ano':   sum_cat['final_prod_ano'].sum(),
    'final_total':      sum_cat['final_total'].sum(),
    'novos_via_produtora': sum_cat['novos_via_produtora'].sum()
}])

glob['conv_total_titulo_%']    = (glob['regra_titulo']   / glob['obras'] * 100).round(2)
glob['conv_titulo→diretor_%']  = (glob['regra_diretor']  / glob['regra_titulo'] * 100).round(2)
glob['conv_titulo→produtora_%']= (glob['regra_produtora']/ glob['regra_titulo'] * 100).round(2)
glob['conv_total_%']           = (glob['final_total']    / glob['obras'] * 100).round(2)

# ---- Exibir ----
fmt = {
    'conv_total_titulo_%':'{:.2f}%',
    'conv_titulo→diretor_%':'{:.2f}%',
    'conv_titulo→produtora_%':'{:.2f}%',
    'conv_total_%':'{:.2f}%'
}
print("Resumo por categoria (com produtora):")
display(sum_cat.style.format(fmt))

print("\nResumo global:")
display(glob.style.format(fmt))


## 5 ) Exports para Excel

In [ ]:
# ================================================================
# ETAPA 1 — Consolidar resultado final e salvar auditoria
# Requer já existirem: matches_dir, matches_prod
# ================================================================

# 1) escolher o nome da coluna CPB conforme sua base (Nº CPB ou N° CPB)
cpb_col = 'Nº CPB' if 'Nº CPB' in filmesBR_SAD_limpo.columns else 'N° CPB'

# 2) união (prioridade ao DIRETOR; o concat mantém a ordem: dir -> prod)
matches_all = pd.concat([matches_dir, matches_prod], ignore_index=True)
matches_all = matches_all.drop_duplicates(subset=['UID', cpb_col], keep='first')

# 3) ordenar e selecionar colunas para visão lado a lado
cols_auditoria = [
    cpb_col, 'UID',
    'Título Original', 'Title', 'BB Original Title', 'Platform Title',
    'Diretor', 'Directors',
    'Ano Inicial', 'Ano Final', 'Year',
    'Produtora', 'BB Primary Company', 'BB Production Companies',
    'categoria', 'match_rule'
]
# mantém colunas que existem; evita KeyError caso alguma não esteja presente
cols_auditoria = [c for c in cols_auditoria if c in matches_all.columns]

matches_all = matches_all[cols_auditoria].sort_values(
    by=['categoria', 'match_rule', 'Title' if 'Title' in matches_all.columns else cpb_col]
).reset_index(drop=True)

# 4) resumos
resumo_categoria = (
    matches_all
      .groupby('categoria')
      .agg(pares=('UID', 'size'),
           uids=('UID', 'nunique'),
           cpbs=(cpb_col, 'nunique'))
      .reset_index()
)

resumo_por_regra = (
    matches_all
      .groupby(['categoria', 'match_rule'])
      .size()
      .unstack(fill_value=0)
      .reset_index()
)

print("Resumo por categoria:")
display(resumo_categoria)

print("\nResumo por regra:")
display(resumo_por_regra)

# 5) salvar Excel de auditoria
arquivo_auditoria = 'Auditoria - matches consolidado.xlsx'
with pd.ExcelWriter(arquivo_auditoria, engine='openpyxl') as writer:
    matches_all.to_excel(writer, sheet_name='consolidado', index=False)
    resumo_categoria.to_excel(writer, sheet_name='resumo_categoria', index=False)
    resumo_por_regra.to_excel(writer, sheet_name='resumo_regra', index=False)

print(f"\nArquivo salvo: {arquivo_auditoria}")


In [ ]:
# ================================================================
# ETAPA 2 — Materializar estágios (por categoria) e exportar
# Requer: build_bb_title_keys, surnames_from_raw, tokens_empresas,
#         _bb_prod_tokens_row, e os DFs *_SAD_limpo / *_BB_limpo
# ================================================================

cpb_col = 'Nº CPB' if 'Nº CPB' in filmesBR_SAD_limpo.columns else 'N° CPB'
ANO_TOL = 2

def materializar_estagios(df_sad, df_bb, categoria):
    # 0) base por título
    bbk = build_bb_title_keys(df_bb)
    sadk = df_sad[[cpb_col, 'Título Original', 'Título Original_processado',
                   'Diretor', 'Produtora', 'Produtora_processada',
                   'Ano Inicial', 'Ano Final']].copy()
    base = bbk.merge(sadk, left_on='BB_title_key', right_on='Título Original_processado', how='inner')

    # estágio 1 — só TÍTULO
    cols_view = [cpb_col, 'BB UID', 'Título Original',
                 'BB Title','BB Original Title','Platform Title',
                 'Diretor','BB Directors',
                 'Produtora','BB Primary Company','BB Production Companies',
                 'Ano Inicial','Ano Final','BB Year']
    titulo_matched = base[cols_view].rename(columns={'BB UID':'UID', 'BB Title':'Title', 'BB Year':'Year'})

    # estágio 2 — TÍTULO + DIRETOR (sem ano)
    a = base['BB Directors'].apply(surnames_from_raw)
    b = base['Diretor'].apply(surnames_from_raw)
    dir_ok = [len(x & y) > 0 for x, y in zip(a, b)]
    titulo_diretor_matched = titulo_matched[dir_ok].copy()

    # estágio 3 — TÍTULO + PRODUTORA (sem ano)
    prod_bb  = base.apply(_bb_prod_tokens_row, axis=1)
    prod_sad = base['Produtora_processada'].apply(tokens_empresas)
    prod_ok  = [len(x & y) > 0 for x, y in zip(prod_bb, prod_sad)]
    titulo_produtora_matched = titulo_matched[prod_ok].copy()

    # helper ano ± tol
    def filtrar_ano(df):
        if df.empty: 
            return df
        df = df.dropna(subset=['Ano Inicial']).copy()
        df['Ano Final'] = df['Ano Final'].fillna(df['Ano Inicial'])
        mask = (
            (df['Year'].astype(int) >= (df['Ano Inicial'].astype(int) - ANO_TOL)) &
            (df['Year'].astype(int) <= (df['Ano Final'].astype(int)   + ANO_TOL))
        )
        return df[mask]

    # estágio 4 — FINAL (título + diretor + ano)
    final_dir_ano  = filtrar_ano(titulo_diretor_matched).copy()

    # estágio 5 — FINAL (título + produtora + ano)
    final_prod_ano = filtrar_ano(titulo_produtora_matched).copy()

    # metadados
    for df in [titulo_matched, titulo_diretor_matched, titulo_produtora_matched, final_dir_ano, final_prod_ano]:
        df['categoria'] = categoria

    return {
        'titulo_matched': titulo_matched,
        'titulo_diretor_matched': titulo_diretor_matched,
        'titulo_produtora_matched': titulo_produtora_matched,
        'final_dir_ano': final_dir_ano,
        'final_prod_ano': final_prod_ano
    }

# ---- rodar para os 4 pares ----
st_filmes_BR  = materializar_estagios(filmesBR_SAD_limpo,  filmesBR_BB_limpo,   'filmes_BR')
st_series_BR  = materializar_estagios(seriesBR_SAD_limpo,  seriesBR_BB_limpo,   'series_BR')
st_filmes_EST = materializar_estagios(filmesBR_SAD_limpo,  filmesEstr_BB_limpo, 'filmes_EST')
st_series_EST = materializar_estagios(seriesBR_SAD_limpo,  seriesEstr_BB_limpo, 'series_EST')

# ---- exportar
arq_estagios = 'Auditoria - estágios por categoria.xlsx'
with pd.ExcelWriter(arq_estagios, engine='openpyxl') as wr:
    for prefixo, st in [
        ('filmes_BR',  st_filmes_BR),
        ('series_BR',  st_series_BR),
        ('filmes_EST', st_filmes_EST),
        ('series_EST', st_series_EST),
    ]:
        st['titulo_matched'].to_excel(wr, sheet_name=f'{prefixo} - título', index=False)
        st['titulo_diretor_matched'].to_excel(wr, sheet_name=f'{prefixo} - título+diretor', index=False)
        st['titulo_produtora_matched'].to_excel(wr, sheet_name=f'{prefixo} - título+prod', index=False)
        st['final_dir_ano'].to_excel(wr, sheet_name=f'{prefixo} - final dir', index=False)
        st['final_prod_ano'].to_excel(wr, sheet_name=f'{prefixo} - final prod', index=False)

print(f'Arquivo salvo: {arq_estagios}')


### 4.3) Elimina plataformas sem interesse em buscas

In [ ]:
# DF completo e coluna de plataforma para o join
df_completo_plataformas = Base_BB
col_plataforma = 'Platform Name'

# Plataformas a excluir (lista)
PLATAFORMAS_EXCLUIR = [
    '99 Media',    'AFA Play',    'ALTBalaji',    'American Indian Film Gallery',    'Amazon Prime Video',    'Anime Onegai',    'AppleTV',     'Apple TV+',    'Archivio Luce',    'AXN',    'BroadwayHD',
    'CINE.AR PLAY',    'Canela.TV',    'Cindie',    'Claro TV+',    'Claro Video',    ' CirqueConnect',    'Combate',    'Crunchyroll',    'Cultpix',    'Curiosity Stream',    'DaFilms',    'DAZN',
    'Dekkoo',    'Demand Africa',    'Digital Concert Hall',    'Digital Theatre',    'Disney+',    'DOCSVILLE',    'Eventive',    'F1 TV',    'Fanatiz',    'FIFA+',    'Filmbox+',    'Filmzie',
    'FlixOlé',    'Globoplay',    'Globe Player',    'GuideDoc',    'HENRI',    'HispanTV',    'History Hit',    'Hoichoi',    'IFI Archive Player',    'IndieFlix',    'iQIYI',    'IWantTFC',    'Kidoodle.TV',
    'KINOA.TV',    'KOCOWA+',    'KweliTV',    'Looke',    'Max',    'MagellanTV',    'Marquee TV',    'Means TV',    'Mercado Play',    'Met Opera on Demand',    'MLB.TV',    'MovieSaints',    'MUBI',
    'NBA League Pass',    'Nebula',    'Netflix',    'OCULTO.TV',    'Oldflix',    'OnDemandKorea',    'OperaVision',    'Paramount+',    'Plex',    'Pluto TV',    'Qello Concerts',    'Rakuten Viki',
    'Reel Short',    'Retina Latina',    'Revry',    'RT en Español',    'Selecta TV',    'ShemarooMe',    'Simply South',    'Sky+',    'Sony Channel',    'Tamanduá TV',    'Teatrix',    'Toon Goggles',
    'Troma NOW!',    'TV Caiçara',    'TVN Play',    'Umbra',    'UNIVER VIDEO',    'Universal+',    'Viddsee',    'Vivo Play',    'Watch',    'WOW Presents Plus',    'YouTube',    'YouTube Premium',    'Zee5',
]

# (opcional) conjunto para checagens/membership rápidas
PLATAFORMAS_EXCLUIR_SET = set(PLATAFORMAS_EXCLUIR)

print("Total plataformas a excluir:", len(PLATAFORMAS_EXCLUIR))


In [ ]:
# ================================================================
# Filas de revisão: estrangeiras com country vazio, diretor presente,
# mapeando plataformas a partir do DF completo (não deduplicado)
# Parâmetros que você PRECISA definir antes:
#   PLATAFORMAS_EXCLUIR = [...]
#   df_completo_plataformas = <seu DF completo, sem deduplicar por UID>
#   col_plataforma = 'Platform Name'  # ajuste se necessário
# ================================================================

# -------- parâmetros do usuário --------
PLATAFORMAS_EXCLUIR
df_completo_plataformas = Base_BB_import
col_plataforma = 'Platform Name'

# -------- checagens explícitas (sem try/except silencioso) --------
if 'BB UID' not in df_completo_plataformas.columns:
    raise KeyError("O DF completo informado NÃO tem a coluna 'BB UID'.")
if col_plataforma not in df_completo_plataformas.columns:
    raise KeyError(f"O DF completo informado NÃO tem a coluna '{col_plataforma}'.")

# -------- mapeia UID -> plataformas (excluindo as indesejadas) --------
f_plat = df_completo_plataformas.copy()

# normaliza plataforma (string) e elimina vazios
f_plat[col_plataforma] = f_plat[col_plataforma].astype(str).str.strip()
f_plat = f_plat[f_plat[col_plataforma] != '']

# remove plataformas indesejadas
mask_keep = ~f_plat[col_plataforma].isin(PLATAFORMAS_EXCLUIR)
f_plat = f_plat[mask_keep]

uid_to_plats = (
    f_plat.groupby('BB UID', as_index=False)[col_plataforma]
          .agg(lambda s: ' | '.join(sorted(set(s))))
          .rename(columns={col_plataforma: 'Plataformas'})
)

# -------- obtém UIDs já casados para excluir da revisão --------
cpb_col = 'Nº CPB' if 'Nº CPB' in filmesBR_SAD_limpo.columns else 'N° CPB'
uids_casados = set(matches_all['UID'].unique())

# -------- função para gerar fila (filmes/séries estrangeiras) --------
def fila_revisao_estr(df_estr, categoria_label):
    if 'BB Countries' not in df_estr.columns:
        raise KeyError(f"O DF '{categoria_label}' não tem a coluna 'BB Countries'.")
    if 'BB Directors' not in df_estr.columns:
        raise KeyError(f"O DF '{categoria_label}' não tem a coluna 'BB Directors'.")

    # country vazio + diretor presente
    base = df_estr[
        (df_estr['BB Countries'].isna()) &
        (df_estr['BB Directors'].astype(str).str.strip() != '')
    ].copy()

    # remove os que já casaram
    base = base[~base['BB UID'].isin(uids_casados)].copy()

    # junta plataformas a partir do DF completo
    out = base.merge(uid_to_plats, on='BB UID', how='left')

    # mantemos apenas quem de fato tem plataforma (após exclusão)
    out = out[out['Plataformas'].notna() & (out['Plataformas'].str.strip() != '')].copy()

    # colunas úteis para revisão
    cols = [c for c in [
        'BB UID', 'BB Title', 'BB Original Title', 'BB Year',
        'BB Directors', 'BB Countries',
        'Plataformas'
    ] if c in out.columns]

    out = out[cols].sort_values(['Plataformas','BB Year','BB Title'], na_position='last')
    out['categoria'] = categoria_label
    return out

# -------- gerar filas para filmes/séries estrangeiras --------
revisao_filmes_EST  = fila_revisao_estr(filmesEstr_BB_limpo, 'filmes_EST')
revisao_series_EST  = fila_revisao_estr(seriesEstr_BB_limpo, 'series_EST')

print("Resumo das filas de revisão:")
print("filmes_EST:", len(revisao_filmes_EST))
print("series_EST:", len(revisao_series_EST))

# -------- exportar para Excel --------
arq_revisao = 'Revisao - estr sem country (por plataformas).xlsx'
with pd.ExcelWriter(arq_revisao, engine='openpyxl') as wr:
    revisao_filmes_EST.to_excel(wr, sheet_name='filmes_EST', index=False)
    revisao_series_EST.to_excel(wr, sheet_name='series_EST', index=False)
print(f"Arquivo salvo: {arq_revisao}")


In [ ]:
# ================================================================
# Listas de NÃO-MATCH para revisão
#  - Estrg (filmes/séries): por plataformas-alvo, diretor presente, não casados
#  - BR   (filmes/séries):  BB Countries contém "BR", não casados
#  Usa Base_BB (completo) para mapear UID -> plataformas e 1º deeplink
# ================================================================

PLATAFORMAS_ALVO = [
    'Filmicca','AmazôniaFLIX','TV Brasil Play','UOL Play',
    'Cine Humberto Mauro Mais','Reserva Imovision','Curta!On','Cinemateca Pernambucana',
]

def _uid_col(df):
    return 'BB UID' if 'BB UID' in df.columns else 'UID'

# ---------------- Checagens mínimas ----------------
for name, df in [
    ('Base_BB_import', Base_BB_import),
    ('filmesBR_BB_limpo', filmesBR_BB_limpo),
    ('seriesBR_BB_limpo', seriesBR_BB_limpo),
    ('filmesEstr_BB_limpo', filmesEstr_BB_limpo),
    ('seriesEstr_BB_limpo', seriesEstr_BB_limpo),
]:
    if _uid_col(df) not in df.columns:
        raise KeyError(f"{name} não tem coluna de UID ('BB UID' ou 'UID').")

if 'Platform Name' not in Base_BB_import.columns or 'Deeplink' not in Base_BB_import.columns:
    raise KeyError("Base_BB_import precisa de 'Platform Name' e 'Deeplink'.")

# ---------------- UIDs já casados (para excluir) ----------------
uids_casados = set(matches_all[_uid_col(matches_all)].unique())

# ---------------- Mapa UID -> plataformas-alvo + 1º deeplink ----------------
uid_full = _uid_col(Base_BB_import)
bb_plat_keep = Base_BB_import[[uid_full,'Platform Name','Deeplink']].copy()
bb_plat_keep = bb_plat_keep[bb_plat_keep['Platform Name'].isin(PLATAFORMAS_ALVO)]

uid_to_plats = (
    bb_plat_keep.sort_values(['Platform Name'])
    .groupby(uid_full, as_index=False)
    .agg({
        'Platform Name': lambda s: ' | '.join(sorted(set(s))),
        'Deeplink': 'first'
    })
    .rename(columns={'Platform Name': 'Plataformas', 'Deeplink': 'Deeplink_1'})
)

# ---------------- Funções canônicas para montar as listas ----------------
BASE_COLS = [
    'BB Title','BB Original Title','BB Year',
    'Seasons','Episodes','Season Numbers',
    'BB Directors','BB Countries',
    'BB Primary Company','BB Production Companies'
]

ORDER_COLS = ['Plataformas','BB Year','BB Title']  # para Estrg
ORDER_COLS_BR = ['BB Year','BB Title']            # para BR

def lista_estrg_por_plataforma(df_estr, categoria_label):
    uidc = _uid_col(df_estr)
    for c in BASE_COLS:
        if c not in df_estr.columns:
            raise KeyError(f"{categoria_label}: coluna '{c}' ausente.")

    base = df_estr.copy()

    # 1) não casados
    base = base[~base[uidc].isin(uids_casados)]

    # 2) DIRETOR OBRIGATÓRIO
    base = base[nonempty_str(base['BB Directors'])]

    # 3) junta plataformas-alvo + deeplink (Left Join; depois mantém só quem achou plataforma-alvo)
    out = base.merge(uid_to_plats, left_on=uidc, right_on=uid_full, how='left')
    out = out[nonempty_str(out['Plataformas'])].copy()

    cols = [uidc] + BASE_COLS + ['Plataformas','Deeplink_1']
    out = out[cols].sort_values(ORDER_COLS, na_position='last')
    out['categoria'] = categoria_label
    return out

def lista_br_sem_match(df_br, categoria_label):
    uidc = _uid_col(df_br)
    for c in BASE_COLS:
        if c not in df_br.columns:
            raise KeyError(f"{categoria_label}: coluna '{c}' ausente.")

    base = df_br.copy()

    # 1) não casados
    base = base[~base[uidc].isin(uids_casados)]

    # 2) Country contém "BR" (casos sem BR já foram para as listas Estrg)
    base = base[base['BB Countries'].astype(str).str.contains('BR', case=False, na=False)]

    out = base.merge(uid_to_plats, left_on=uidc, right_on=uid_full, how='left')
    cols = [uidc] + BASE_COLS + ['Plataformas','Deeplink_1']
    out = out[cols].sort_values(ORDER_COLS_BR, na_position='last')
    out['categoria'] = categoria_label
    return out

# ---------------- Materialização (4 listas) ----------------
estrg_filmes = lista_estrg_por_plataforma(filmesEstr_BB_limpo, 'Estrg Filmes')
estrg_series = lista_estrg_por_plataforma(seriesEstr_BB_limpo, 'Estrg Séries')
br_filmes    = lista_br_sem_match(filmesBR_BB_limpo, 'BR Filmes')
br_series    = lista_br_sem_match(seriesBR_BB_limpo, 'BR Séries')

# Diagnóstico rápido
print("Resumo (linhas):",
      f"Estrg Filmes={len(estrg_filmes)} | Estrg Séries={len(estrg_series)} | BR Filmes={len(br_filmes)} | BR Séries={len(br_series)}")

# ---------------- Export ----------------
arq_listas = "Revisao - NaoMatch (Estrg por plataforma + BR).xlsx"
with pd.ExcelWriter(arq_listas, engine='openpyxl') as wr:
    estrg_filmes.to_excel(wr, sheet_name='Estrg Filmes', index=False)
    estrg_series.to_excel(wr, sheet_name='Estrg Séries', index=False)
    br_filmes.to_excel(wr,    sheet_name='BR Filmes',    index=False)
    br_series.to_excel(wr,    sheet_name='BR Séries',    index=False)
print(f"Arquivo salvo: {arq_listas}")

## 5) Separação de listas de buscas

### 5.1) Importação dos resultados de match 2023


### 5.2) Separação dos dataframes de obras brasileiras das bases BB

### 5.3) Modificação das dfs de obras brasileiras para gerar as listas 

### 5.4) Criação das listas individuais

In [ ]:
# ================================================================
# Listas finais de NÃO-MATCH para revisão manual
#  - Estrg (filmes/séries): sem CPB, país vazio, diretor presente,
#    e presentes nas plataformas foco (whitelist)
#  - BR   (filmes/séries): sem CPB e BB Countries contendo "BR"
#  Join por BB UID na Base_BB_import para trazer Plataformas e Deeplink
# ================================================================

PLATAFORMAS_FOCO = [
    'Filmicca',
    'AmazôniaFLIX',
    'TV Brasil Play',
    'UOL Play',
    'Cine Humberto Mauro Mais',
    'Reserva Imovision',
    'Curta!On',
    'Cinemateca Pernambucana',
]

def _uidcol(df):
    return 'BB UID' if 'BB UID' in df.columns else 'UID'

# UIDs já casados (para excluir)
uids_casados = set(matches_all[_uidcol(matches_all)].unique())

# Mapa UID -> plataformas whitelist + 1º deeplink
uid_full = _uidcol(Base_BB_import)
plat_map = (
    Base_BB_import[[uid_full, 'Platform Name', 'Deeplink']]
    .query("`Platform Name` in @PLATAFORMAS_FOCO")
    .sort_values(['Platform Name'])
    .groupby(uid_full, as_index=False)
    .agg({'Platform Name': lambda s: ' | '.join(sorted(set(s))),
          'Deeplink': 'first'})
    .rename(columns={'Platform Name':'Plataformas', 'Deeplink':'Deeplink_1'})
)

def _montar_lista(df_bb,
                  precisa_br=None,            # True -> contém 'BR'; False -> vazio/na; None -> ignora
                  precisa_diretor=False,
                  filtrar_plataforma=False,    # True em Estrg
                  categoria=''):
    uid = _uidcol(df_bb)

    base = df_bb.copy()
    base = base[~base[uid].isin(uids_casados)]

    if precisa_br is True:
        base = base[base['BB Countries'].astype(str).str.contains('BR', case=False, na=False)]
    elif precisa_br is False:
        base = base[~base['BB Countries'].astype(str).str.contains('BR', case=False, na=False)]
        base = base[base['BB Countries'].isna() | (base['BB Countries'].astype(str).str.strip() == '')]

    if precisa_diretor:
        base = base[base['BB Directors'].astype(str).str.strip() != '']

    if filtrar_plataforma:
        base = base.merge(plat_map, left_on=uid, right_on=uid_full, how='inner')
    else:
        # só para exibição (não filtra por plataforma)
        base = base.merge(plat_map, left_on=uid, right_on=uid_full, how='left')

    final_cols = [
        uid, 'Platform Title', 'BB Original Title', 'BB Title', 'BB Year',
        'Seasons', 'Episodes', 'Season Numbers',
        'BB Directors', 'BB Primary Company', 'BB Production Companies',
        'BB Countries', 'Plataformas', 'Deeplink_1'
    ]
    final_cols = [c for c in final_cols if c in base.columns]

    out = base[final_cols].sort_values(['Plataformas','BB Year','BB Title'], na_position='last')
    out['categoria'] = categoria

    # Campos para trabalho manual (vêm vazios)
    for c in ['Responsável', 'Nº CPB encontrado', 'Título encontrado', 'Observação']:
        out[c] = pd.NA
    return out

# -------- 4 listas finais --------
filmes_ESTR = _montar_lista(filmesEstr_BB_limpo, precisa_br=False, precisa_diretor=True,
                            filtrar_plataforma=True,  categoria='filmes_ESTR')
series_ESTR = _montar_lista(seriesEstr_BB_limpo, precisa_br=False, precisa_diretor=True,
                            filtrar_plataforma=True,  categoria='series_ESTR')
filmes_BR   = _montar_lista(filmesBR_BB_limpo,   precisa_br=True,  precisa_diretor=False,
                            filtrar_plataforma=False, categoria='filmes_BR')
series_BR   = _montar_lista(seriesBR_BB_limpo,   precisa_br=True,  precisa_diretor=False,
                            filtrar_plataforma=False, categoria='series_BR')

# Sanidade: interseção deve ser 0
u_estr = set(filmes_ESTR[_uidcol(filmes_ESTR)]) | set(series_ESTR[_uidcol(series_ESTR)])
u_br   = set(filmes_BR[_uidcol(filmes_BR)])     | set(series_BR[_uidcol(series_BR)])
print("Interseção (UIDs em ESTR e BR):", len(u_estr & u_br))

# Exporta
arq = "Revisao - NaoMatch (Estrg por plataforma + BR) — FINAL.xlsx"
with pd.ExcelWriter(arq, engine='openpyxl') as wr:
    filmes_BR.to_excel(wr,   sheet_name='filmes_BR',   index=False)
    series_BR.to_excel(wr,   sheet_name='series_BR',   index=False)
    filmes_ESTR.to_excel(wr, sheet_name='filmes_ESTR', index=False)
    series_ESTR.to_excel(wr, sheet_name='series_ESTR', index=False)
print("Arquivo salvo:", arq)


In [ ]:
print('Estrg Filmes com diretor vazio:', (estrg_filmes['BB Directors'].isna() | (estrg_filmes['BB Directors'].astype(str).str.strip() == '')).sum())
print('Estrg Séries com diretor vazio:', (estrg_series['BB Directors'].isna() | (estrg_series['BB Directors'].astype(str).str.strip() == '')).sum())


In [ ]:
# pega a base "estrangeira" ANTES do merge
base_fe = filmesEstr_BB_limpo.copy()   # idem para séries depois
uidc = 'BB UID' if 'BB UID' in base_fe.columns else 'UID'

# quantos candidatos totais após excluir casados?
tmp = base_fe[~base_fe[uidc].isin(uids_casados)]

# validação de diretor com uma limpeza mais rígida (remove espaços invisíveis também)
def _trim_hard(s):
    s = s.astype(str).str.replace(r'[\u00A0\u2007\u202F\u200B-\u200D]', '', regex=True)  # NBSP, thin spaces, zero-width
    s = s.str.strip()
    return s

dir_raw  = tmp['BB Directors']
dir_trim = _trim_hard(dir_raw)

print('1A) candidatos pós-exclusão de casados:', len(tmp))
print('1B) com diretor VAZIO (após trim hard):',
      ((dir_trim.isna()) | (dir_trim == '') | (dir_trim.str.lower().isin(['nan','none']))).sum())

# amostra de 10 problemáticos (se houver)
bad_idx = ((dir_trim.isna()) | (dir_trim == '') | (dir_trim.str.lower().isin(['nan','none'])))
display(tmp.loc[bad_idx, [uidc,'BB Title','BB Directors']].head(10))


In [ ]:
# reconstrói a lista como no pipeline, mas guardando intermediários
base_ok = tmp[_trim_hard(tmp['BB Directors']).ne('') & tmp['BB Directors'].notna()]
out = base_ok.merge(uid_to_plats, left_on=uidc, right_on=_uid_col(Base_BB_import), how='left')
out = out[out['Plataformas'].astype(str).str.strip().ne('') & out['Plataformas'].notna()].copy()

# checagem pós-merge
dir_after = _trim_hard(out['BB Directors'])
print('2) após merge/filtro plataforma, diretor vazio =',
      ((dir_after.isna()) | (dir_after == '') | (dir_after.str.lower().isin(['nan','none']))).sum())

# se >0, mostra 10 linhas para inspecionar
bad2 = ((dir_after.isna()) | (dir_after == '') | (dir_after.str.lower().isin(['nan','none'])))
display(out.loc[bad2, [uidc,'BB Title','BB Directors','Plataformas']].head(10))


In [ ]:
# usa exatamente o DF que você está salvando no Excel (ex.: estrg_filmes)
print('estrg_filmes (linhas):', len(estrg_filmes))

# validação direta no DF final
df_final = estrg_filmes.copy()
dtrim = df_final['BB Directors'].astype(str).replace(r'[\u00A0\u2007\u202F\u200B-\u200D]', '', regex=True).str.strip()
print('3) no DF final, diretor vazio =', ((dtrim=='') | dtrim.isna() | dtrim.str.lower().isin(['nan','none'])).sum())

# Exporta CSV temporário só com as linhas "suspeitas", se houver
sus = df_final[((dtrim=='') | dtrim.isna() | dtrim.str.lower().isin(['nan','none']))]
sus.to_csv('suspeitos_diretor_vazio.csv', index=False, encoding='utf-8')
print('CSV com suspeitos salvo (se não estiver vazio): suspeitos_diretor_vazio.csv')


In [ ]:
print('linhas a exportar:', len(estrg_filmes))
print('diretor vazio na exportação:',
      (estrg_filmes['BB Directors'].isna() |
       (estrg_filmes['BB Directors'].astype(str).str.strip() == '')).sum())